In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
%matplotlib inline
import networkx as nx 
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt 
%matplotlib inline 

N=2;S0=1

r = 0.75;a=-0.5;b=2

p = (r-a)/(b-a)
q = (b-r)/(b-a)

def plot_tree(g):
    plt.figure(figsize=(20,10))
    pos={}
    lab={}    
    for n in g.nodes():
        pos[n]=(n[0],n[1])
        lab[n]=float("{0:.2f}".format(g.node[n]['value']))
    elarge=g.edges(data=True)
    nx.draw_networkx_edges(g,pos,edgelist=elarge)
    nx.draw_networkx_labels(g,pos,lab,font_size=15,font_family='sans-serif')
    plt.ylim(-N+0.5,N+1.5) 
    plt.xlim(-0.5,N+0.5)
    plt.show()
    
def graph_stock():
    S=nx.Graph()
    for k in range(0,N):
        for l in range(-k+1,k+3,2):
            S.add_edge((k,l),(k+1,l+1))
            S.add_edge((k,l),(k+1,l-1))
            
    for n in S.nodes():
        k=n[0]
        l=n[1]-1
        S.node[n]['value']=S0*((1.0+b)**((k+l)/2))*((1.0+a)**((k-l)/2))
    return S

plot_tree(graph_stock())

In [ ]:
def European_call_price(K):

    price = nx.Graph()    
    hedge = nx.Graph()
    S = graph_stock()

    for k in range(0,N):
            for l in range(-k+1,k+3,2):
                price.add_edge((k,l),(k+1,l+1))
                price.add_edge((k,l),(k+1,l-1))
                hedge.add_edge((k,l),(k+1,l+1))
                hedge.add_edge((k,l),(k+1,l-1))
    
    for l in range(-N+1,N+3,2):
        price.node[(N,l)]['value'] = np.maximum(S.node[(N,l)]['value']-K,0)
     
    for k in reversed(range(0,N)):
        for l in range(-k+1,k+3,2):
            price.node[(k,l)]['value'] = (price.node[(k+1,l+1)]['value']*p+price.node[(k+1,l-1)]['value']*q)/(1+r)        
    return price

In [ ]:
K = input("Strike K=")

call_price = European_call_price(float(K))

print('Underlying asset prices:')
plot_tree(graph_stock())
print('European call prices:')
plot_tree(call_price)
print('Price at time 0 of the European call option:',float("{0:.4f}".format(call_price.node[(0,1)]['value'])))

In [ ]:
def European_put_price(K):

    price = nx.Graph()    
    S = graph_stock()

    for k in range(0,N):
            for l in range(-k+1,k+3,2):
                price.add_edge((k,l),(k+1,l+1))
                price.add_edge((k,l),(k+1,l-1))
    
    for l in range(-N+1,N+3,2):
        price.node[(N,l)]['value'] = np.maximum(K-S.node[(N,l)]['value'],0)
     
    for k in reversed(range(0,N)):
        for l in range(-k+1,k+3,2):
            price.node[(k,l)]['value'] = (price.node[(k+1,l+1)]['value']*p+price.node[(k+1,l-1)]['value']*q)/(1+r)        
    return price

In [ ]:
def European_put_parity(K):

    price = nx.Graph()    
    call = European_call_price(K)
    S = graph_stock()

    for k in range(0,N):
            for l in range(-k+1,k+3,2):
                price.add_edge((k,l),(k+1,l+1))
                price.add_edge((k,l),(k+1,l-1))

    for k in reversed(range(0,N+1)):
        for l in range(-k+1,k+3,2):
            price.node[(k,l)]['value'] = call.node[(k,l)]['value'] + K*(1.0+r)**(k-N) - S.node[(k,l)]['value']
    return price

In [ ]:
K = input("Strike=")

put_price = European_put_price(float(K))
put_parity = European_put_parity(float(K))

print('Underlying asset prices:')
plot_tree(graph_stock())
print('European put prices:')
plot_tree(put_price)
print('European put parity prices:')
plot_tree(put_parity)
print('Price at time 0 of the European put option:',float("{0:.4f}".format(put_price.node[(0,1)]['value'])))

In [ ]:
def European_call_hedge(K):

    price = European_call_price(K)
    hedge = nx.Graph()
    S = graph_stock()

    for k in range(0,N):
            for l in range(-k+1,k+3,2):
                hedge.add_edge((k,l),(k+1,l+1))
                hedge.add_edge((k,l),(k+1,l-1))
    
    for l in range(-N+1,N+3,2):
        hedge.node[(N,l)]['value'] = 0

    for k in reversed(range(0,N)):
        for l in range(-k+1,k+3,2):
            hedge.node[(k,l)]['value'] = (price.node[(k+1,l+1)]['value']-price.node[(k+1,l-1)]['value'])/(b-a)/(S.node[(k,l)]['value'])
    return hedge

In [ ]:
K = input("Strike K=")

call_price = European_call_price(float(K))

print('Underlying asset prices:')
plot_tree(graph_stock())
print('European call prices:')
plot_tree(European_call_price(float(K)))
print('Price at time 0 of the European call option:',float("{0:.4f}".format(call_price.node[(0,1)]['value'])))
print('Hedging strategy:')
plot_tree(European_call_hedge(float(K)))